In [1]:
# Librerías
import pandas as pd
import numpy as np
from pathlib import Path
import os

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
sns.set_theme(style="whitegrid")

In [3]:
# Variables
src_path = os.getcwd()
data_path = "/data/raw/"
data_file = "merged_data.csv"

file_path= Path(os.path.dirname(src_path)+data_path+data_file)

In [5]:
df = pd.read_csv(file_path)

In [7]:
df.shape

(1339, 44)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1339 entries, 0 to 1338
Data columns (total 44 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             1339 non-null   int64  
 1   Species                1339 non-null   object 
 2   Owner                  1332 non-null   object 
 3   Country.of.Origin      1338 non-null   object 
 4   Farm.Name              980 non-null    object 
 5   Lot.Number             276 non-null    object 
 6   Mill                   1021 non-null   object 
 7   ICO.Number             1182 non-null   object 
 8   Company                1130 non-null   object 
 9   Altitude               1113 non-null   object 
 10  Region                 1280 non-null   object 
 11  Producer               1107 non-null   object 
 12  Number.of.Bags         1339 non-null   int64  
 13  Bag.Weight             1339 non-null   object 
 14  In.Country.Partner     1339 non-null   object 
 15  Harv

<h2 style="text-align: center;">ANALIZANDO EL DATASET</h1>

El dataset consta de 1339 filas por 44 columnas y contiene la información relativa a diferentes variedades de café de tipo Arabica y Robusta revisada por personal del CQI (Coffee Quality Institute).

La información se divide en:

Calidad de café (Cupping Score)
- Aroma
- Flavor
- Aftertaste
- Acidity
- Body
- Balance
- Uniformity
- Clean Cup
- Sweetness


Semilla (Bean)
- Species (arabica / robusta)
- Variety
- Processing Method
- Moisture
- Color
- Category.One.Defects
- Category.Two.Defects
- Quakers


Plantación (Farm)
- Country.of.Origin
- Farm.Name
- Mill
- Owner
- Company
- Altitude
- Region



La columna "Total Cup Points" será nuestro medidor de calidad, siendo mejor el café cuanto más alta sea esta puntuación.

Realizaremos un análisis univariante y multivariante de todas las variables, con el objetivo final de encontrar un algoritmo que permita predecir qué café tendrá una mejor puntuación en función de sus características.


#### Cleaning data

Deleting columns we don't need to analyze: "Unnamed", "ICO Number", "Certification Contact", "Certification Address" and we will use ID as index

In [ ]:
df = df.drop(columns=["Unnamed: 0", "ICO Number", "Certification Contact", "Certification Address"]).set_index("ID")

In [ ]:
# Duplicates
df[df.duplicated()]

No duplicates. Can go on with the analysis.

In [ ]:
# Unique values
for column in df.columns:
    unique_val = df[column].unique()
    print("Unique values for: {}: {}".format(column, len(df[column].unique())))

In [ ]:
# Null / Missing values
df[df.columns[df.isnull().any()]].isnull().sum().sort_values(ascending=False)

All the columns with missing values are object type so we replace all nan values with UNKNOWN.

In [ ]:
df.fillna("UNKNOWN", inplace=True)

### UNIVARIANT ANALYSIS

#### COUNTRY OF ORIGIN

In [ ]:
df["Country of Origin"].unique()

No need to translate or change values in any way, so we can proceed with the analysis

In [ ]:
sns.countplot(data=df, y="Country of Origin")

#### Checking values for VARIETY

In [ ]:
df["Variety"].unique()

There are multiple and different values for unknown varieties or missing values. We will standarize it and assign the value "UNKNOWN" for all of them.

Also, we will replace the '+' symbols as well as 'and' or 'y' words with commas to correctly separate the coffee varieties.

To finish, we will write all the words in capital letters.

In [ ]:
df["Variety"][(df["Variety"]=='unknown') | (df["Variety"]=='unknow')] = "UNKNOWN"
df.loc[df["Variety"].str.contains(' \+ '), 'Variety'] = df["Variety"].str.replace(' + ',',')
df.loc[df["Variety"].str.contains('\+'), 'Variety'] = df["Variety"].str.replace('+',',')
df.loc[df["Variety"].str.contains(' and '), 'Variety'] = df["Variety"].str.replace(' and ',',')
df.loc[df["Variety"].str.contains(' Y '), 'Variety'] = df["Variety"].str.replace(' Y ',',')
df.loc[df["Variety"].str.contains(' & '), 'Variety'] = df["Variety"].str.replace(' & ',',')
df["Variety"] = df["Variety"].str.upper()


#### Checking values for PROCESSING METHOD

In [ ]:
df["Processing Method"].unique()

We replace the method in spanish for the translated one, which is "Semi Washed" and set all values to capital letters

In [ ]:
df["Processing Method"][(df["Processing Method"]=='SEMI-LAVADO')] = 'Semi Washed'
df["Processing Method"] = df["Processing Method"].str.upper()